# Дашборд для Яндекс.Дзен

## Описание проекта

Вы работаете аналитиком в Яндекс.Дзене. Почти всё ваше время занимает анализ пользовательского взаимодействия с карточками статей.

Каждую карточку определяют её тема и источник (у него тоже есть тема). Примеры тем: «Красота и здоровье», «Россия», «Путешествия». А пользователей системы характеризует возрастная категория.

Есть три способа взаимодействия пользователей с системой:

- Карточка отображена для пользователя (show);
- Пользователь кликнул на карточку (click);
- Пользователь просмотрел статью карточки (view).
- Каждую неделю начинающие менеджеры Денис и Валерия задают вам одни и те же вопросы:

1. Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
2. Как много карточек генерируют источники с разными темами?
3. Как соотносятся темы карточек и темы источников?

На шестую неделю работы вы решаете, что процесс пора автоматизировать. Для Дениса и Валерии нужно сделать дашборд.

Дашборд будет основываться на пайплайне, который будет брать данные из таблицы, в которых хранятся сырые данные, трансформировать данные и укладывать их в агрегирующую таблицу. Пайплайн будет разработан для вас дата-инженерами.

## Краткое ТЗ для создания дашборда:

***Бизнес-задача***: анализ взаимодействия пользователей с карточками Яндекс.Дзен;
***Насколько часто предполагается пользоваться дашбордом***: не реже, чем раз в неделю;
***Кто будет основным пользователем дашборда***: менеджеры по анализу контента;

***Состав данных для дашборда:***

- История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
- Разбивка событий по темам источников;
- Таблица соответствия тем источников темам карточек;

***По каким параметрам данные должны группироваться:***

- Дата и время;
- Тема карточки;
- Тема источника;
- Возрастная группа;
- Характер данных:

История событий по темам карточек — абсолютные величины с разбивкой по минутам;
Разбивка событий по темам источников — относительные величины (% событий);
Соответствия тем источников темам карточек - абсолютные величины;

***Важность:*** все графики имеют равную важность;
Источники данных для дашборда:

Сырые данные о событиях взаимодействия пользователей с карточками (таблица dash_visits);
База данных, в которой будут храниться агрегированные данные:

Дополнительные агрегированные таблицы в БД zen;

***Частота обновления данных:***
Один раз в сутки, в полночь по UTC;

## Задачи проекта

1. Получить данные из базы данных

2. Создать дашборд на основе полученных данных

3. Ответить на вопросы менеджеров в виде презентации:

- Cколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?

- Как много карточек генерируют источники с разными темами?

- Как соотносятся темы карточек и темы источников?

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import Image

***Макет дашборда:***

In [2]:
Image(url='https://pictures.s3.yandex.net/resources/Untitled_-_2020-07-06T160925.436_1594041010.png')

In [3]:
#коннектимся к базе
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

#запрашиваем сырые данные
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

In [4]:
# делаем sql запрос

query = '''
           SELECT *
           FROM dash_visits
        '''

dash_visits = pd.io.sql.read_sql(query, con=engine)

In [5]:
dash_visits.head(5) 

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


In [6]:
# посмотрим основную информацию
print(dash_visits.info())
print('\n')
print(dash_visits.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB
None


          record_id        visits
count  3.074500e+04  30745.000000
mean   1.055969e+06     10.089673
std    8.875461e+03     19.727601
min    1.040597e+06      1.000000
25%    1.048283e+06      1.000000
50%    1.055969e+06      3.000000
75%    1.063655e+06     10.000000
max    1.071341e+06    371.000000


In [7]:
# проверим на явные дубликаты
dash_visits.duplicated().sum()

0

In [8]:
# посмотрим на темы карточек 
dash_visits['item_topic'].value_counts()

Отношения             1536
Интересные факты      1535
Наука                 1505
Подборки              1456
Полезные советы       1424
Общество              1422
Россия                1385
История               1363
Семья                 1287
Путешествия           1247
Деньги                1234
Женщины               1230
Дети                  1229
Туризм                1206
Здоровье              1203
Красота               1193
Культура              1160
Юмор                  1129
Искусство             1119
Рассказы              1109
Психология            1056
Скандалы              1023
Знаменитости           976
Женская психология     914
Шоу                    804
Name: item_topic, dtype: int64

In [9]:
# посмотрим на темы источников
dash_visits['source_topic'].value_counts()

Семейные отношения    1822
Россия                1687
Знаменитости          1650
Полезные советы       1578
Путешествия           1563
Кино                  1505
Дети                  1459
История               1437
Семья                 1405
Одежда                1379
Здоровье              1243
Искусство             1228
Авто                  1077
Психология            1055
Сад и дача            1036
Политика              1024
Спорт                 1007
Сделай сам             995
Ремонт                 985
Деньги                 973
Еда                    912
Интерьеры              809
Строительство          758
Музыка                 750
Технологии             741
Финансы                667
Name: source_topic, dtype: int64

In [10]:
# посмотрим на возрастные группы
dash_visits['age_segment'].value_counts()

18-25    7056
26-30    5875
31-35    5552
36-40    5105
41-45    3903
45+      3254
Name: age_segment, dtype: int64

In [11]:
# посмотрим за какой период получены данные
print(dash_visits['dt'].min(), '||', dash_visits['dt'].max())

2019-09-24 18:28:00 || 2019-09-24 19:00:00


In [12]:
# преобразуем в csv
dash_visits.to_csv('dash_visits.csv', index = False)

# Дашборд: 

https://public.tableau.com/app/profile/alexander4727/viz/yandex-dzen_Vorosh/Dashboard1

# Презентация:

https://disk.yandex.ru/i/PsBlLUAYWG_X0w

# Общий вывод:

Дубликатов, пропусков и других ошибок не обнаружено.

Для построенния дашборда получили данные по посещениям карточек Яндекс.Дзена за период с 18:28:00 по 19:00:00, 24 сентября 2019 года.

Построив дашборд мы можем отметить следующие посещения карточек:

На самый пик событий приходится суммарно 61.247 посещений.

***Топ-5 наиболее популярных тем карточек:***

- Наука (4.372 посещения)
- Отношения (4.145 посещений)
- Интересные факты (3.910 посещений)
- Общество (3.723 посещений)
- Подборки (3.520 посещений)

***Топ-5 наименее популярных тем карточек:***

- Знаменитости (1.463 посещения)
- Шоу (1.485 посещений)
- Женская психология (1.518 посещений)
- Психология (1.538 посещений)
- Искусство (1.707 посещений)

***По теме источников генерируют следующие количества посещенений:***

Топ-5 наиболее популярных тем источников:
- Семейные отношения (10-11% посещений)
- Россия (9-10% посещений)
- Полезные советы (8-9% посещений)
- Путешествия (7-8% посещений)
- Знаменитости (7-8% посещений)

Топ-5 наименее популярных тем источников:
- Финансы (0.8-1% посещений)
- Музыка (0.8-1% посещений)
- Строительство (0.8-1% посещений)
- Технологии (1% посещений)
- Интерьеры (1.24% посещений)

***Топ-5 наиболее популярных связок «источник - тема карточки»:***
- Путешествия - Рассказы (4.587 посещений)
- Россия - Общество (3.471 посещение)
- Кино - Наука (3.279 посещений)
- Россия - Россия (2.847 посещений)
- Полезные советы - Подборки (2.795 посещений)